In [ ]:
import numpy as np
from sympy.physics.wigner import wigner_3j,wigner_6j
import sympy as sy
from numpy import linalg as LA
from IPython.display import Latex,display
import matplotlib.pyplot as plt
from matplotlib import colors
%matplotlib inline
import seaborn as sns
import ujson as uj
#sns.set()
#sns.set_palette('bright')
from Energy_Levels import MoleculeLevels
from Energy_Levels import branching_ratios, Calculate_TDMs,Calculate_TDM_evecs, Calculate_forbidden_TDM_evecs, Calculate_forbidden_TDMs
from functools import partial
#np.set_printoptions(precision=9, suppress=True)
# from tabulate import tabulate
from matplotlib.collections import LineCollection
from scipy.stats import norm

In [ ]:
X010_173 = MoleculeLevels.initialize_state('YbOH','173','X010',[1,2],M_values = 'all',I=[5/2,1/2],S=1/2,round=8,P_values=[1/2,3/2])

In [ ]:
evals_173, evecs_173 = X010_173.eigensystem(0,1e-8,order=True,method='torch')

In [ ]:
out = X010_173.calculate_two_photon_spectrum(40, 1e-8,list(range(84)), parity_sign = 1, laser_polarization = 'orth')

In [ ]:
transitions = out[1]

# Frequency bounds (AOM 0 frequency)
freq_min = 98
freq_max = 101

#offset  = 8.184 # MHz

#offset  = 0*8.384 # MHz
offset  = (106.089-97.58)*2 # MHz

# Filter transitions within bounds
filtered_transitions = [t for t in transitions if (freq_min <= (t['freq']-offset)/2 -90 <= freq_max)]  


# Spectrum parameters
freq_range = np.linspace(freq_min, freq_max, 1000)  # Define the frequency range for the plot
freq_width = 0.02 # Gaussian width in MHz

# Initialize the spectrum
spectrum = np.zeros_like(freq_range)

# Create figure with constrained layout
fig, ax = plt.subplots(figsize=(12, 8), constrained_layout=True)

for transition in filtered_transitions:
    freq = (transition['freq']- offset)/2 -90   #accounting for 1 MHz offset correction
    label = f"({transition['state index 0']}→{transition['state index 1']})"
    
    # Generate Gaussian peak
    gaussian_peak = norm.pdf(freq_range, loc=freq, scale=freq_width)
    spectrum += gaussian_peak  # Add the peak to the spectrum
    
    # Plot the peak
    ax.plot(freq_range, gaussian_peak, label=f"Peak {label}")
    
    
    # Annotate the peak with state information
    if transition['M0'] >= 0:
        ax.text(freq, max(gaussian_peak) * 1.1, f"M0={transition['M0']}, {transition['state index 0']}\nM1={transition['M1']}, {transition['state index 1']}\nf={round(freq,3)}",
            fontsize=12, ha='center')
    
# Plot the combined spectrum
ax.plot(freq_range, spectrum, color='black', linewidth=2, label='Spectrum')

# Labels and title
ax.set_xlabel('Frequency (MHz)')
ax.set_ylabel('Intensity (Arbitrary Units)')
ax.set_title(f'Spectrum with Gaussian Peaks (Freq Range: {freq_min}-{freq_max} MHz)')
ax.grid(True)
ax.legend()

# Show the plot
plt.show()
